<a href="https://colab.research.google.com/github/ShimriFernando/MyCaptainAI-ML/blob/master/ML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy 
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [2]:
file = open("Frankenstein-21.txt").read()

In [3]:
def tokenize_words(input):
    input = input.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

processed_inputs = tokenize_words(file)

In [4]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c,i)for i, c in enumerate(chars))

In [5]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print("Total number of characters:", input_len)
print("Total vocab:", vocab_len)

Total number of characters: 3906
Total vocab: 29


In [6]:
seq_length = 100
x_data = []
y_data = []
for i in range(0, input_len - seq_length, 1):
    in_seq = processed_inputs[i:i + seq_length]
    out_seq = processed_inputs[i + seq_length]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])
n_patterns = len(x_data)
print("Total Patterns:", n_patterns)

Total Patterns: 3806


In [7]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [8]:
y = np_utils.to_categorical(y_data)

In [9]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [12]:
model.fit(X,y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
3806/3806 [==============================] - 9s 2ms/step - loss: 3.0758

Epoch 00001: loss improved from inf to 3.07578, saving model to model_weights_saved.hdf5
Epoch 2/4
3806/3806 [==============================] - 7s 2ms/step - loss: 2.9452

Epoch 00002: loss improved from 3.07578 to 2.94523, saving model to model_weights_saved.hdf5
Epoch 3/4
3806/3806 [==============================] - 7s 2ms/step - loss: 2.9379

Epoch 00003: loss improved from 2.94523 to 2.93786, saving model to model_weights_saved.hdf5
Epoch 4/4
3806/3806 [==============================] - 7s 2ms/step - loss: 2.9241

Epoch 00004: loss improved from 2.93786 to 2.92410, saving model to model_weights_saved.hdf5


In [13]:
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
num_to_char = dict((i, c) for i,c in enumerate(chars))


In [17]:
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print( "\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" especially remained several years child continue narrative must record incident took place four year "


In [20]:
for i in range(100):
    x = numpy.reshape(pattern, (1,len(pattern), 1))
    x = x/float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]